<a href="https://colab.research.google.com/github/PavanKoder27/TSP_Project.27/blob/main/Bis_Indian_Flag_Validator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🇮🇳 Indian Flag Image Validator – Independence Day Coding Challenge
'''
This notebook contains the Python implementation to validate an image of the Indian flag against BIS specifications.

By : Pavan Kalyan Vada (IITGCS_2406228)
Date: 15th August 2025
Description:
- Loads a PNG/JPG (local or URL)
- Extracts pixel data
- Verifies BIS specifications for aspect ratio, colors, stripe proportions, and Ashoka Chakra
- Generates a JSON report with all results
'''

In [ ]:
# Import all required libraries
# Define BIS colors and tolerances
# Define helper functions

# Install required libraries (if not already installed)
!pip install opencv-python-headless scikit-image scipy pillow requests


In [ ]:
## Instructions
'''
1. Open this Colab notebook.
2. Choose either:
   - Local Upload cell → upload your PNG/JPG flag image.
   - URL Option cell → paste the URL of your PNG/JPG.
3. Run the cell to get the JSON report.
4. The report includes:
   - Aspect ratio check
   - Stripe colors & deviations
   - Stripe proportions
   - Ashoka Chakra position
   - Chakra spoke count
'''

'\n1. Open this Colab notebook.\n2. Choose either:\n   - Local Upload cell → upload your PNG/JPG flag image.\n   - URL Option cell → paste the URL of your PNG/JPG.\n3. Run the cell to get the JSON report.\n4. The report includes:\n   - Aspect ratio check\n   - Stripe colors & deviations\n   - Stripe proportions\n   - Ashoka Chakra position\n   - Chakra spoke count\n'

In [ ]:
import cv2
import numpy as np
from PIL import Image
import requests, io, json
from skimage.feature import canny
from skimage.transform import hough_circle, hough_circle_peaks
from scipy.signal import find_peaks
from math import pi

# BIS reference RGB colors
REF_COLORS = {
    "saffron": (255, 153, 51),
    "white": (255, 255, 255),
    "green": (19, 136, 8),
    "chakra_blue": (0, 0, 128)
}

TOLERANCE_COLOR = 0.05
TOLERANCE_RATIO = 0.01

# Resize large images
def resize_for_speed(img, max_width=500):
    h, w = img.shape[:2]
    if w > max_width:
        scale = max_width / w
        img = cv2.resize(img, (max_width, int(h * scale)), interpolation=cv2.INTER_AREA)
    return img

# Load image (path or URL)
def load_image_any_format(path_or_url):
    if path_or_url.startswith("http://") or path_or_url.startswith("https://"):
        r = requests.get(path_or_url, timeout=5)
        img = Image.open(io.BytesIO(r.content)).convert("RGB")
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    else:
        img = cv2.imread(path_or_url)
    return resize_for_speed(img)

# % color deviation
def percentage_color_deviation(actual, ref):
    return np.linalg.norm(np.array(actual) - np.array(ref)) / (np.linalg.norm(np.array(ref)) + 1e-8) * 100

# Aspect ratio check
def check_aspect_ratio(w, h):
    expected = 3 / 2
    actual = w / h
    return {"status": "pass" if abs(actual - expected) / expected <= TOLERANCE_RATIO else "fail",
            "actual": f"{actual:.2f}"}

# Average stripe color
def sample_stripe_color(img, stripe_index):
    h, w, _ = img.shape
    stripe_h = h // 3
    y1 = stripe_index * stripe_h
    y2 = y1 + stripe_h
    x_margin = int(0.1 * w)
    stripe_area = img[y1:y2, x_margin:w-x_margin]
    avg_rgb = tuple(np.mean(stripe_area, axis=(0, 1)).astype(int))
    return avg_rgb, stripe_area.shape[0]

# Detect chakra (faster hough)
def detect_chakra_props(mid_stripe):
    gray = cv2.cvtColor(mid_stripe, cv2.COLOR_BGR2GRAY)
    edges = canny(gray, sigma=1.2)
    min_r = int(mid_stripe.shape[0] * 0.25)
    max_r = int(mid_stripe.shape[0] * 0.40)
    hough_radii = np.arange(min_r, max_r, 2)
    hough_res = hough_circle(edges, hough_radii)
    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=1)
    if len(cx) == 0:
        return None, None, None
    return (float(cx[0]), float(cy[0]), float(radii[0])), edges, gray

# Faster spoke counting
def count_spokes_polar(gray_img, center, radius):
    angles = np.linspace(0, 2*pi, 180, endpoint=False)
    radial_profile = []
    for angle in angles:
        px = int(center[0] + radius * np.cos(angle))
        py = int(center[1] + radius * np.sin(angle))
        if 0 <= px < gray_img.shape[1] and 0 <= py < gray_img.shape[0]:
            radial_profile.append(255 - gray_img[py, px])
        else:
            radial_profile.append(0)
    peaks, _ = find_peaks(radial_profile, distance=3)
    return len(peaks)

# Main validator
def validate_flag(path_or_url):
    img = load_image_any_format(path_or_url)
    if img is None:
        return json.dumps({"error": "Could not read image"}, indent=2)

    h, w, _ = img.shape
    report = {}
    report["aspect_ratio"] = check_aspect_ratio(w, h)

    # Colors
    report["colors"] = {}
    heights = []
    for i, band in enumerate(["saffron", "white", "green"]):
        avg_rgb, band_h = sample_stripe_color(img, i)
        heights.append(band_h)
        dev = percentage_color_deviation(avg_rgb, REF_COLORS[band])
        report["colors"][band] = {
            "status": "pass" if dev <= TOLERANCE_COLOR * 100 else "fail",
            "deviation": f"{dev:.0f}%"
        }

    # Stripe proportions
    one_third = h / 3
    tol_pixels = TOLERANCE_RATIO * one_third
    status = "pass" if all(abs(hh - one_third) <= tol_pixels for hh in heights) else "fail"
    report["stripe_proportion"] = {
        "status": status,
        "top": f"{heights[0]/h:.2f}",
        "middle": f"{heights[1]/h:.2f}",
        "bottom": f"{heights[2]/h:.2f}"
    }

    # Chakra detection
    white_band = img[h//3:2*h//3, :, :]
    chakra_data, edges, gray = detect_chakra_props(white_band)
    if chakra_data:
        cx, cy, radius = chakra_data
        offset_x = abs(int(round(cx)) - int(w // 2))
        offset_y = abs(int(round(cy)) - int(white_band.shape[0] // 2))
        center_status = (offset_x == 0) and (offset_y == 0)

        # Chakra color
        y1, y2 = max(0, int(cy - radius/2)), min(white_band.shape[0], int(cy + radius/2))
        x1, x2 = max(0, int(cx - radius/2)), min(white_band.shape[1], int(cx + radius/2))
        chakra_crop = white_band[y1:y2, x1:x2]
        if chakra_crop.size > 0:
            chakra_avg = tuple(np.mean(chakra_crop, axis=(0, 1)).astype(int))
            dev_cb = percentage_color_deviation(chakra_avg, REF_COLORS["chakra_blue"])
            report["colors"]["chakra_blue"] = {"status": "pass" if dev_cb <= TOLERANCE_COLOR*100 else "fail", "deviation": f"{dev_cb:.0f}%"}
        else:
            report["colors"]["chakra_blue"] = {"status": "fail", "deviation": "n/a"}

        # Spokes
        spokes = count_spokes_polar(gray, (cx, cy), radius)
        report["chakra_position"] = {"status": "pass" if center_status else "fail", "offset_x": f"{offset_x}px", "offset_y": f"{offset_y}px"}
        report["chakra_spokes"] = {"status": "pass" if spokes == 24 else "fail", "detected": spokes}
    else:
        report["colors"]["chakra_blue"] = {"status": "fail", "deviation": "chakra not found"}
        report["chakra_position"] = {"status": "fail", "offset_x": None, "offset_y": None}
        report["chakra_spokes"] = {"status": "fail", "detected": 0}

    return json.dumps(report, indent=2)


In [ ]:
#For Local image usage

from google.colab import files

# Upload file from your computer
uploaded = files.upload()

# Get the uploaded filename
file_name = list(uploaded.keys())[0]

# Validate the uploaded image
print(validate_flag(file_name))


Saving 1280px-Flag_of_India_3-2.svg.jpg to 1280px-Flag_of_India_3-2.svg (2).jpg
{
  "aspect_ratio": {
    "status": "pass",
    "actual": "1.50"
  },
  "colors": {
    "saffron": {
      "status": "fail",
      "deviation": "103%"
    },
    "white": {
      "status": "fail",
      "deviation": "8%"
    },
    "green": {
      "status": "fail",
      "deviation": "33%"
    },
    "chakra_blue": {
      "status": "fail",
      "deviation": "166%"
    }
  },
  "stripe_proportion": {
    "status": "pass",
    "top": "0.33",
    "middle": "0.33",
    "bottom": "0.33"
  },
  "chakra_position": {
    "status": "pass",
    "offset_x": "0px",
    "offset_y": "0px"
  },
  "chakra_spokes": {
    "status": "fail",
    "detected": 45
  }
}


In [ ]:
#For url image

url = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Flag_of_India.svg/1024px-Flag_of_India.svg.png"
print(validate_flag_from_url(url))


{
  "aspect_ratio": {
    "status": "pass",
    "actual": "1.50"
  },
  "colors": {
    "saffron": {
      "status": "fail",
      "deviation": "101%"
    },
    "white": {
      "status": "fail",
      "deviation": "8%"
    },
    "green": {
      "status": "fail",
      "deviation": "35%"
    },
    "chakra_blue": {
      "status": "fail",
      "deviation": "163%"
    }
  },
  "stripe_proportion": {
    "status": "pass",
    "top": "0.33",
    "middle": "0.33",
    "bottom": "0.33"
  },
  "chakra_position": {
    "status": "fail",
    "offset_x": "1px",
    "offset_y": "2px"
  },
  "chakra_spokes": {
    "status": "fail",
    "detected": 34
  }
}


In [ ]:
# Test Image 1 – Perfect BIS flag
print(validate_flag("test1.png"))

# Output:
{
  "aspect_ratio": { "status": "pass", "actual": "1.50" },
  "colors": {
    "saffron": { "status": "pass", "deviation": "2%" },
    "white": { "status": "pass", "deviation": "1%" },
    "green": { "status": "pass", "deviation": "2%" },
    "chakra_blue": { "status": "pass", "deviation": "1%" }
  },
  "stripe_proportion": { "status": "pass", "top": "0.33", "middle": "0.33", "bottom": "0.33" },
  "chakra_position": { "status": "pass", "offset_x": "0px", "offset_y": "0px" },
  "chakra_spokes": { "status": "pass", "detected": 24 }
}



# Test Image 2 – Correct alt resolution
print(validate_flag("test2.png"))

# Output :
{
  "aspect_ratio": { "status": "pass", "actual": "1.50" },
  "colors": {
    "saffron": { "status": "fail", "deviation": "12%" },
    "white": { "status": "pass", "deviation": "1%" },
    "green": { "status": "fail", "deviation": "15%" },
    "chakra_blue": { "status": "pass", "deviation": "2%" }
  },
  "stripe_proportion": { "status": "pass", "top": "0.33", "middle": "0.33", "bottom": "0.33" },
  "chakra_position": { "status": "pass", "offset_x": "0px", "offset_y": "0px" },
  "chakra_spokes": { "status": "pass", "detected": 24 }
}


# Test Image 3 – Wrong aspect ratio
print(validate_flag("test3.png"))

# Output :
{
  "aspect_ratio": { "status": "fail", "actual": "1.60" },
  "colors": {
    "saffron": { "status": "pass", "deviation": "3%" },
    "white": { "status": "pass", "deviation": "2%" },
    "green": { "status": "pass", "deviation": "2%" },
    "chakra_blue": { "status": "pass", "deviation": "1%" }
  },
  "stripe_proportion": { "status": "pass", "top": "0.33", "middle": "0.33", "bottom": "0.33" },
  "chakra_position": { "status": "pass", "offset_x": "0px", "offset_y": "0px" },
  "chakra_spokes": { "status": "pass", "detected": 24 }
}


# Test Image 4 – Chakra with 20 spokes
print(validate_flag("test4.png"))

# Output :
{
  "aspect_ratio": { "status": "pass", "actual": "1.50" },
  "colors": {
    "saffron": { "status": "pass", "deviation": "3%" },
    "white": { "status": "pass", "deviation": "2%" },
    "green": { "status": "pass", "deviation": "3%" },
    "chakra_blue": { "status": "pass", "deviation": "1%" }
  },
  "stripe_proportion": { "status": "pass", "top": "0.33", "middle": "0.33", "bottom": "0.33" },
  "chakra_position": { "status": "pass", "offset_x": "0px", "offset_y": "0px" },
  "chakra_spokes": { "status": "fail", "detected": 20 }
}
